# Deep Learning for NLP - Project

RULES:

* Do not create any additional cell

* Fill in the blanks

* All cells should be runnable (modulo trivial compatibility bugs that we'd fix)

* 4 / 20 points will be allocated to the clarity of your code

* Efficient code will have a bonus

DELIVERABLE:

* this notebook
* the predictions of the SST test set

DO NOT INCLUDE THE DATASETS IN THE DELIVERABLE..

In [1]:
import io
import os
import numpy as np
import scipy

In [2]:
PATH_TO_DATA = "data/"

# 1) Monolingual (English) word embeddings 

In [127]:
class Word2vec():
    def __init__(self, fname, nmax=100000):
        self.load_wordvec(fname, nmax)
        self.word2id = dict.fromkeys(self.word2vec.keys())
        self.id2word = {v: k for k, v in self.word2id.items()}
        self.embeddings = np.array(self.word2vec.values())
    
    def load_wordvec(self, fname, nmax):
        self.word2vec = {}
        with io.open(fname, encoding='utf-8') as f:
            next(f)
            for i, line in enumerate(f):
                word, vec = line.split(' ', 1)
                self.word2vec[word] = np.fromstring(vec, sep=' ')
                if i == (nmax - 1):
                    break
        print('Loaded %s pretrained word vectors' % (len(self.word2vec)))

    def most_similar(self, w, K=5):
        keys_word = list(self.word2vec.keys())
        scores_data = np.zeros(len(keys_word))
        
        for ind, word in enumerate(keys_word):
            scores_data[ind] = self.score(w, word)
            
        inds_words = np.argsort(scores_data)[-K:]
        words_close = np.array(keys_word)[inds_words]
        # K most similar words: self.score  -  np.argsort 
        return(words_close)

    def score(self, w1, w2):
        vec1 = self.word2vec[w1]
        vec2 = self.word2vec[w2]
        
        score_cosine = vec1.dot(vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2))
        # cosine similarity: np.dot  -  np.linalg.norm
        return(score_cosine)


In [ ]:
w2v = Word2vec(os.path.join(PATH_TO_DATA, 'crawl-300d-200k.vec'), nmax=100000)

# You will be evaluated on the output of the following:
for w1, w2 in zip(('cat', 'dog', 'dogs', 'paris', 'germany'), ('dog', 'pet', 'cats', 'france', 'berlin')):
    print(w1, w2, w2v.score(w1, w2))
for w1 in ['cat', 'dog', 'dogs', 'paris', 'germany']:
    print(w2v.most_similar(w1))

In [128]:
class BoV():
    def __init__(self, w2v):
        self.w2v = w2v
    
    def encode(self, sentences, idf=False):
        # takes a list of sentences, outputs a numpy array of sentence embeddings
        # see TP1 for help
        sentemb = []
        for sent in sentences:
            if idf is False:
                vect = []
                for ind, w in enumerate(sent):
                    try:
                        vect.append(self.w2v.word2vec[w])
                    except:
                        print(w)
                vect = np.vstack(vect)
                sentemb.append(np.mean(vect, axis = 0))
                # mean of word vectors
            else:
                vect = []
                for ind, w in enumerate(sent):
                    try:
                        vect.append(self.w2v.word2vec[w]*idf[w])
                    except:
                        print(w)
                vect = np.vstack(vect)
                sentemb.append(np.mean(vect, axis = 0))
                # idf-weighted mean of word vectors
        return np.vstack(sentemb)

    def most_similar(self, s, sentences, idf=False, K=5):
        # get most similar sentences and **print** them
        keys = self.encode(sentences, idf)
        query = self.encode([s], idf)
        
        scores_data = np.zeros(len(keys))
        for ind, sent in enumerate(keys):
            scores_data[ind] = self.score(np.array(query), np.array(sent), idf)
            
        inds_sent = np.argsort(scores_data)[-K:]
        sent_close = np.array(sentences)[inds_sent]
        print('phrases proches', sent_close)
        # K most similar words: self.score  -  np.argsort
        return(sent_close)

    def score(self, s1, s2, idf=False):
        if type(s1) == list:
            s1 = np.array(self.encode([s1], idf))
            s2 = np.array(self.encode([s2], idf)).transpose()
        score_cosine = s1.dot(s2)/(np.linalg.norm(s1)*np.linalg.norm(s2))
        # cosine similarity: use   np.dot  and  np.linalg.norm
        return(score_cosine)
    
    def build_idf(self, sentences):
        # build the idf dictionary: associate each word to its idf value
        idf = {}
        for sent in sentences:
            for w in set(sent):
                idf[w] = idf.get(w, 0) + 1
        
        for word in idf.keys():
            idf[word] = max(1, np.log10(len(sentences) / (idf[word])))
            
        return(idf)

In [ ]:
w2v = Word2vec(os.path.join(PATH_TO_DATA, 'crawl-300d-200k.vec'), nmax=10000)
s2v = BoV(w2v)

# Load sentences in "PATH_TO_DATA/sentences.txt"
sentences = [] #list of list of words
with io.open(os.path.join(PATH_TO_DATA, 'sentences.txt'), encoding='utf-8') as f:
        for line in f:
            sentences.append(line.split())
            
# Build idf scores for each word
s2v.build_idf(sentences)
idf = {} if True else s2v.build_idf(sentences)

# You will be evaluated on the output of the following:
s2v.most_similar('' if not sentences else sentences[10], sentences)  # BoV-mean
print(s2v.score('' if not sentences else sentences[7], '' if not sentences else sentences[13]))


idf = s2v.build_idf(sentences)
s2v.most_similar('' if not sentences else sentences[10], sentences, idf)  # BoV-idf
print(s2v.score('' if not sentences else sentences[7], '' if not sentences else sentences[13], idf))

# 2) Multilingual (English-French) word embeddings

Let's consider a bilingual dictionary of size V_a (e.g French-English).

Let's define **X** and **Y** the **French** and **English** matrices.

They contain the embeddings associated to the words in the bilingual dictionary.

We want to find a **mapping W** that will project the source word space (e.g French) to the target word space (e.g English).

Procrustes : **W\* = argmin || W.X - Y ||  s.t  W^T.W = Id**
has a closed form solution:
**W = U.V^T  where  U.Sig.V^T = SVD(Y.X^T)**

In what follows, you are asked to: 

In [62]:
# 1 - Download and load 50k first vectors of
#     https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.en.vec
#     https://s3-us-west-1.amazonaws.com/fasttext-vectors/wiki.fr.vec

# TYPE CODE HERE

def load_wordvec(fname, nmax):
        word2vec = {}
        with io.open(fname, encoding='utf-8') as f:
            next(f)
            for i, line in enumerate(f):
                word, vec = line.split(' ', 1)
                word2vec[word] = np.fromstring(vec, sep=' ')
                if i == (nmax - 1):
                    break
        print('Loaded %s pretrained word vectors' % (len(word2vec)))
        return(word2vec)
        
french_vec = os.path.join(PATH_TO_DATA, 'wiki.fr.vec')
engl_vec = os.path.join(PATH_TO_DATA, 'wiki.en.vec')
d = 300
nmax = 100000
french_w2v = load_wordvec(french_vec, nmax)
engl_w2v = load_wordvec(engl_vec, nmax)

Loaded 100000 pretrained word vectors
Loaded 100000 pretrained word vectors


In [27]:
# 2 - Get words that appear in both vocabs (= identical character strings)
#     Use it to create the matrix X and Y (of aligned embeddings for these words)

# TYPE CODE HERE
def intersection_linear_complex(lst1, lst2): 
    temp = set(lst2) 
    lst3 = [value for value in lst1 if value in temp] 
    return(lst3)

french_words = [*french_w2v.keys()]
english_words = [*engl_w2v.keys()]

common_words = intersection_linear_complex(french_words, english_words)

french_common = np.zeros((d, len(common_words)))
english_common = np.zeros((d, len(common_words)))
for ind, word in enumerate(common_words):
    french_common[:,ind] = french_w2v[word]
    english_common[:,ind] = engl_w2v[word]
    
french = np.zeros((d, len(french_words)))
english = np.zeros((d, len(english_words)))
ind_to_french = {}
ind_to_english = {}
for ind, word in enumerate(french_words):
    french[:,ind] = french_w2v[word]
    ind_to_french[ind] = word
for ind, word in enumerate(english_words): 
    english[:,ind] = engl_w2v[word]
    ind_to_english[ind] = word
    

In [33]:
# 3 - Solve the Procrustes using the scipy package and: scipy.linalg.svd() and get the optimal W
#     Now W*French_vector is in the same space as English_vector

# TYPE CODE HERE
import scipy.linalg
U, s, Vh = scipy.linalg.svd(english_common.dot(french_common.transpose()), full_matrices=False)

W_opt = U.dot(Vh)


(300, 18970) (300, 300) (300, 300) (300, 300) (300, 300)


In [43]:
# 4 - After alignment with W, give examples of English nearest neighbors of some French words (and vice versa)
#     You will be evaluated on that part and the code above

# TYPE CODE HERE
from scipy import spatial

english_hat = W_opt.dot(french)
french_hat = (W_opt.transpose()).dot(english)
words_french = ['ici', 'amis', 'mais', 'crayon', 'aide', 'sanctuaire']
words_english = ['doctor', 'she', 'why', 'london','friend','friends']

print("English to French")
for word in words_english:
    dist_english_hat = scipy.spatial.distance_matrix(english_hat.transpose(), engl_w2v[word].reshape((1,-1)))
    word_closest = np.argmin(dist_english_hat)
    print(word + " : " + ind_to_french[word_closest])

print("")
print("Français à l'anglais")
for word in words_french:
    dist_french_hat = scipy.spatial.distance_matrix(french_hat.transpose(), french_w2v[word].reshape((1,-1)))
    word_closest = np.argmin(dist_french_hat)
    print(word + " : " + ind_to_english[word_closest])
    

English to French
doctor : docteur
she : elle
why : pourquoi
london : londres
friend : ami
friends : amis
Français à l'anglais
ici : here
amis : friends
mais : but
crayon : pencil
aide : help
sanctuaire : shrine
(300, 50000) (300, 50000)


If you want to dive deeper on this subject: https://github.com/facebookresearch/MUSE

# 3) Sentence classification with BoV and scikit-learn

In [89]:
# 1 - Load train/dev/test of Stanford Sentiment TreeBank (SST)
#     (https://nlp.stanford.edu/~socherr/EMNLP2013_RNTN.pdf)

def load_Bov(fname):
        Bov_nb = []
        Bov_sent = []
        with io.open(fname, encoding='utf-8') as f:
            for i, line in enumerate(f):
                words = line.split(' ')
                words[-1] = words[-1].strip()
                if words[0].isdigit():
                    Bov_nb.append(words[0])
                    Bov_sent.append(words[1:])
                else:
                    Bov_sent.append(words)
        print('Loaded %s pretrained word vectors' % (len(Bov_sent)))
        return(Bov_nb, Bov_sent)
    
train_BoV_file = os.path.join(PATH_TO_DATA, 'SST', 'stsa.fine.train')
dev_BoV_file = os.path.join(PATH_TO_DATA, 'SST', 'stsa.fine.dev')
test_BoV_file = os.path.join(PATH_TO_DATA, 'SST', 'stsa.fine.test.X')

train_sentences_list = load_Bov(train_BoV_file)
dev_sentences_list = load_Bov(dev_BoV_file)
test_sentences_list = load_Bov(test_BoV_file)


Loaded 8544 pretrained word vectors
Loaded 1101 pretrained word vectors
Loaded 2210 pretrained word vectors


In [163]:
# 2 - Encode sentences with the BoV model above

def compute_matrixes(sentences_list, w2v):
    nb_sentences = len(sentences_list[1])
    BoV_sent = -1*np.ones(d)
    sentences_array = -1*np.ones((nb_sentences,d))
    nb_array = -1*np.ones(nb_sentences)
    unknown_words = []
    for ind_sent in range(nb_sentences):
        #for each sentence
        nb_word = 0
        for word in sentences_list[1][ind_sent]:
            #for each word in the sentence
            if word in w2v.keys():
                BoV_sent += w2v[word]
                nb_word += 1
            else:
                unknown_words.append(word)
        sentences_array[ind_sent,:] = BoV_sent/nb_word
        if len(sentences_list[0]) > ind_sent:
            nb_array[ind_sent] = sentences_list[0][ind_sent]
    return(sentences_array, nb_array)

sentences_train_array, nb_train_array = compute_matrixes(train_sentences_list, engl_w2v)
sentences_dev_array, nb_dev_array = compute_matrixes(dev_sentences_list, engl_w2v)
sentences_test_array, nb_test_array = compute_matrixes(test_sentences_list, engl_w2v)

In [165]:
# 3 - Learn Logistic Regression on top of sentence embeddings using scikit-learn
#     (consider tuning the L2 regularization on the dev set)

from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial', C=0.05).fit(sentences_train_array, nb_train_array)

np.sum(clf.predict(sentences_dev_array) == nb_dev_array)

238

In [166]:
# 4 - Produce 2210 predictions for the test set (in the same order). One line = one prediction (=0,1,2,3,4).
#     Attach the output file "logreg_bov_y_test_sst.txt" to your deliverable.
#     You will be evaluated on the results of the test set.

clf.predict(sentences_test_array)

array([3., 3., 3., ..., 2., 2., 2.])

In [ ]:
# BONUS!
# 5 - Try to improve performance with another classifier
#     Attach the output file "XXX_bov_y_test_sst.txt" to your deliverable (where XXX = the name of the classifier)

# TYPE CODE HERE

# 4) Sentence classification with LSTMs in Keras

## 4.1 - Preprocessing

In [92]:
import keras

In [93]:
# 1 - Load train/dev/test sets of SST
train_BoV_file = os.path.join(PATH_TO_DATA, 'SST', 'stsa.fine.train')
dev_BoV_file = os.path.join(PATH_TO_DATA, 'SST', 'stsa.fine.dev')
test_BoV_file = os.path.join(PATH_TO_DATA, 'SST', 'stsa.fine.test.X')

train_sentences_list = load_Bov(train_BoV_file)
dev_sentences_list = load_Bov(dev_BoV_file)
test_sentences_list = load_Bov(test_BoV_file)

Loaded 8544 pretrained word vectors
Loaded 1101 pretrained word vectors
Loaded 2210 pretrained word vectors


In [111]:
# 2 - Transform text to integers using keras.preprocessing.text.one_hot function
#     https://keras.io/preprocessing/text/

string_sentences_list = ' '.join(str(r) for v in train_sentences_list[1] for r in v)

set_words = set(keras.preprocessing.text.text_to_word_sequence(string_sentences_list))
train_size = len(set_words)
train_size = 5000

def sentences_list_to_int_sentences_list(sentences_list, train_size):
    string_sentences_list = []
    int_sentences_list = []
    for v in sentences_list[1]:
        string_sent = ' '.join(v)
        string_sentences_list.append(string_sent)
        int_sentences_list.append(keras.preprocessing.text.one_hot(string_sent, train_size, lower=True, split=" ")) #filters=base_filter(), 
    return(int_sentences_list)

int_train_sentences_list = sentences_list_to_int_sentences_list(train_sentences_list, train_size)
int_dev_sentences_list = sentences_list_to_int_sentences_list(dev_sentences_list, train_size)
int_test_sentences_list = sentences_list_to_int_sentences_list(test_sentences_list, train_size)

**Padding input data**

Models in Keras (and elsewhere) take batches of sentences of the same length as input. It is because Deep Learning framework have been designed to handle well Tensors, which are particularly suited for fast computation on the GPU.

Since sentences have different sizes, we "pad" them. That is, we add dummy "padding" tokens so that they all have the same length.

The input to a Keras model thus has this size : (batchsize, maxseqlen) where maxseqlen is the maximum length of a sentence in the batch.

In [112]:
# 3 - Pad your sequences using keras.preprocessing.sequence.pad_sequences
#     https://keras.io/preprocessing/sequence/

#array_train_sentences = keras.preprocessing.sequence.pad_sequences(int_train_sentences_list, maxlen=32, dtype='int32')
#array_dev_sentences = keras.preprocessing.sequence.pad_sequences(int_dev_sentences_list, maxlen=32, dtype='int32')
#array_test_sentences = keras.preprocessing.sequence.pad_sequences(int_test_sentences_list, maxlen=32, dtype='int32')
array_glob_sentences = keras.preprocessing.sequence.pad_sequences(int_train_sentences_list+int_dev_sentences_list, maxlen=15, dtype='int32')
array_train_sentences = array_glob_sentences[:len(int_train_sentences_list)]
array_dev_sentences = array_glob_sentences[len(int_train_sentences_list):]


def compute_train_set(sentences_list):
    array_nb = np.zeros((len(sentences_list[0]), 5))
    for ind, classe in enumerate(sentences_list[0]):
        array_nb[ind, int(classe)] = 1
    return(array_nb)

array_train_nb = compute_train_set(train_sentences_list)
array_dev_nb = compute_train_set(dev_sentences_list)

## 4.2 - Design and train your model

In [119]:
# 4 - Design your encoder + classifier using keras.layers
#     In Keras, Torch and other deep learning framework, we create a "container" which is the Sequential() module.
#     Then we add components to this contained : the lookuptable, the LSTM, the classifier etc.
#     All of these components are contained in the Sequential() and are trained together.

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Activation

embed_dim  = 15  # word embedding dimension
nhid       = 20  # number of hidden units in the LSTM
vocab_size = train_size  # size of the vocabulary
n_classes  = 5

model = Sequential()
model.add(Embedding(vocab_size, embed_dim))
model.add(LSTM(nhid, dropout_W=0.2, dropout_U=0.2))
model.add(Dense(n_classes, activation='sigmoid'))


C:\Users\anatole parre\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(20, dropout=0.2, recurrent_dropout=0.2)`
  app.launch_new_instance()


In [120]:
# 5 - Define your loss/optimizer/metrics

# MODIFY CODE BELOW

loss_classif     =  'categorical_crossentropy' # find the right loss for multi-class classification
optimizer        =  'adam' # find the right optimizer
metrics_classif  =  ['accuracy']

# Observe how easy (but blackboxed) this is in Keras
model.compile(loss=loss_classif,
              optimizer=optimizer,
              metrics=metrics_classif)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, None, 15)          75000     
_________________________________________________________________
lstm_16 (LSTM)               (None, 20)                2880      
_________________________________________________________________
dense_16 (Dense)             (None, 5)                 105       
Total params: 77,985
Trainable params: 77,985
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# 6 - Train your model and find the best hyperparameters for your dev set
#     you will be evaluated on the quality of your predictions on the test set

# ADAPT CODE BELOW
bs = 64
n_epochs = 10

x_train = array_train_sentences
y_train = array_train_nb
x_val = array_dev_sentences
y_val = array_dev_nb

history = model.fit(x_train, y_train, batch_size=bs, nb_epoch=n_epochs, validation_data=(x_val, y_val))

C:\Users\anatole parre\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  del sys.path[0]


Train on 8544 samples, validate on 1101 samples
Epoch 1/10
8544/8544 [==============================] - 9s 1ms/step - loss: 1.5811 - acc: 0.2711 - val_loss: 1.5692 - val_acc: 0.2534
Epoch 2/10
5376/8544 [=================>............] - ETA: 1s - loss: 1.5574 - acc: 0.2863

In [ ]:
# 7 - Generate your predictions on the test set using model.predict(x_test)
#     https://keras.io/models/model/
#     Log your predictions in a file (one line = one integer: 0,1,2,3,4)
#     Attach the output file "logreg_lstm_y_test_sst.txt" to your deliverable.

# TYPE CODE HERE


## 4.3 -- innovate !

In [ ]:
# 8 - Open question: find a model that is better on your dev set
#     (e.g: use a 1D ConvNet, use a better classifier, pretrain your lookup tables ..)
#     you will get point if the results on the test set are better: be careful of not overfitting your dev set too much..
#     Attach the output file "XXX_XXX_y_test_sst.txt" to your deliverable.

# TYPE CODE HERE


In [52]:
# 1 - Load train/dev/test of Stanford Sentiment TreeBank (SST)
#     (https://nlp.stanford.edu/~socherr/EMNLP2013_RNTN.pdf)
"""
def find_parens(s):
    toret = {}
    pstack = []
    nbtoret = {}
    for i, c in enumerate(s):
        if c == '(':
            pstack.append(i)
        elif c == ')':
            if len(pstack) == 0:
                raise IndexError("No matching closing parens at: " + str(i))
            ind = pstack.pop()
            toret[ind] = i
            nbtoret[ind] = int(s[ind+1])
    if len(pstack) > 0:
        raise IndexError("No matching opening parens at: " + str(pstack.pop()))

    return(toret, nbtoret)

def load_Bov(fname, nmax):
        Bov = []
        with io.open(fname, encoding='utf-8') as f:
            next(f)
            for i, line in enumerate(f):
                toret, nbtoret = find_parens(line)
                wordtoret = {}
                sent_to_nb = {}
                for key in toret.keys():
                    part_line = line[key+3:toret[key]]
                    if part_line[0] == '(':
                        #then there is more than one word
                        part_line = part_line.split(' ')
                        sent = []
                        for word in part_line:
                            if word[-1] == ')':
                                #then we have a word : 'exemple)' and not a number '(2'
                                word_without_parenth = word.split(')')[0]
                                sent.append(word_without_parenth.lower())
                        wordtoret[key] = sent
                    else:
                        #there is only one word
                        wordtoret[key] = [part_line.lower()]
                for key in toret.keys():
                    sent_to_nb[key] = [nbtoret[key], wordtoret[key]]
                Bov.append(sent_to_nb)
                #if i == (nmax - 1):
                #    break
        print('Loaded %s pretrained word vectors' % (len(Bov)))
        return(Bov)
    
train_BoV_file = os.path.join(PATH_TO_DATA, 'trees', 'train.txt')
nmax = 10
train_BoV = load_Bov(train_BoV_file, nmax)
"""

'\ndef find_parens(s):\n    toret = {}\n    pstack = []\n    nbtoret = {}\n    for i, c in enumerate(s):\n        if c == \'(\':\n            pstack.append(i)\n        elif c == \')\':\n            if len(pstack) == 0:\n                raise IndexError("No matching closing parens at: " + str(i))\n            ind = pstack.pop()\n            toret[ind] = i\n            nbtoret[ind] = int(s[ind+1])\n    if len(pstack) > 0:\n        raise IndexError("No matching opening parens at: " + str(pstack.pop()))\n\n    return(toret, nbtoret)\n\ndef load_Bov(fname, nmax):\n        Bov = []\n        with io.open(fname, encoding=\'utf-8\') as f:\n            next(f)\n            for i, line in enumerate(f):\n                toret, nbtoret = find_parens(line)\n                wordtoret = {}\n                sent_to_nb = {}\n                for key in toret.keys():\n                    part_line = line[key+3:toret[key]]\n                    if part_line[0] == \'(\':\n                        #then there 